# ClipCard Field Evaluation: DevOps/SRE

**Purpose:** This notebook analyzes historical data from the `clipmap` CSVs to evaluate the real-world performance of the ClipCard system over time for the **DevOps/SRE** silo.

It calculates and visualizes key metrics as defined in the `dfm-devops-sre.md` framework document, such as:
- MTTR
- Change Failure Rate
- Deployment Frequency

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Load the aggregated ClipCard data
try:
    cards_df = pd.read_csv('../../data/clipmap/cards.csv')
    events_df = pd.read_csv('../../data/clipmap/events.csv')
    # Filter for the correct domain and merge
    devops_cards = cards_df[cards_df['domain'] == 'dfm-devops-sre'].copy()
    merged_df = pd.merge(devops_cards, events_df, left_on='id', right_on='card_id')
    merged_df['time'] = pd.to_datetime(merged_df['time'])
except FileNotFoundError as e:
    print(f'ERROR: {e.filename} not found. Please ensure the data exists.')
    merged_df = pd.DataFrame()

### Metric 1: Change Failure Rate

**Note:** The sample data does not contain enough detail to calculate a true `change_failure_rate`. We will use a placeholder calculation based on the number of incidents prevented vs. successful changes, grouped by week.

In [ ]:
if not merged_df.empty:
    events_by_week = merged_df.set_index('time')
    
    # Placeholder for failures - we'll use 'Incident Prevented' as a proxy
    weekly_failures = events_by_week[events_by_week['outcome'] == 'Incident Prevented'].resample('W').size()
    weekly_total = events_by_week.resample('W').size()
    
    cfr_df = pd.DataFrame({'failures': weekly_failures, 'total': weekly_total}).fillna(0)
    # Calculate rate per 100 deployments as per the DFM
    cfr_df['cfr'] = (cfr_df['failures'] / cfr_df['total'] * 100).fillna(0)

    # Plotting
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(cfr_df.index, cfr_df['cfr'], marker='o', linestyle='-', label='Change Failure Rate (%)')

    ax.set_title('Change Failure Rate Over Time (Placeholder)')
    ax.set_ylabel('Rate per 100 Deployments')
    ax.set_xlabel('Week')
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.xticks(rotation=45)
    ax.legend()
    plt.tight_layout()
    plt.show()
else:
    print('Data is empty. Cannot calculate Change Failure Rate.')